In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
from tensorflow.keras.applications import resnet50
from tensorflow.keras.optimizers import Adam

# ---------------------------------------------------
# 1. Load the oxford_flowers102 Dataset
# ---------------------------------------------------
# Create train/validation split
(dataset_train, dataset_test), info = tfds.load(
    'oxford_flowers102',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True
)

# Number of classes in oxford_flowers102
num_classes = info.features['label'].num_classes

# ---------------------------------------------------
# 2. Preprocess Data for ResNet
# ---------------------------------------------------
IMG_SIZE = 224
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

def preprocess_resnet(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = resnet50.preprocess_input(image)  # Use resnet50's preprocess_input
    return image, tf.one_hot(label, num_classes)

# Create tf.data pipelines
train_ds_resnet = (dataset_train
                   .map(preprocess_resnet, num_parallel_calls=AUTOTUNE)
                   .batch(batch_size)
                   .prefetch(AUTOTUNE))

test_ds_resnet = (dataset_test
                  .map(preprocess_resnet, num_parallel_calls=AUTOTUNE)
                  .batch(batch_size)
                  .prefetch(AUTOTUNE))

# ---------------------------------------------------
# 3. Define ResNet50 Model
# ---------------------------------------------------
def create_resnet(input_shape, num_classes):
    base_model = resnet50.ResNet50(weights="imagenet", include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model

    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inputs=base_model.input, outputs=outputs, name="ResNet50")
    return model

# ---------------------------------------------------
# 4. Compile and Train the Model
# ---------------------------------------------------
# ResNet50 Model
resnet_model = create_resnet((IMG_SIZE, IMG_SIZE, 3), num_classes)
resnet_model.compile(optimizer=Adam(learning_rate=1e-4), loss="categorical_crossentropy", metrics=["accuracy"])

print("\n--- Fine-Tuning ResNet50 on oxford_flowers102 ---")
history_resnet = resnet_model.fit(train_ds_resnet, epochs=50, validation_data=test_ds_resnet, verbose=1)

# Evaluate ResNet50
loss_resnet, acc_resnet = resnet_model.evaluate(test_ds_resnet, verbose=0)
print(f"ResNet50 Final Accuracy: {acc_resnet:.4f}")



--- Fine-Tuning ResNet50 on oxford_flowers102 ---
Epoch 1/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 29s 721ms/step - accuracy: 0.0059 - loss: 5.5271 - val_accuracy: 0.0245 - val_loss: 4.8869
Epoch 2/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 22s 121ms/step - accuracy: 0.0371 - loss: 4.9157 - val_accuracy: 0.0392 - val_loss: 4.5852
Epoch 3/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 144ms/step - accuracy: 0.0504 - loss: 4.5453 - val_accuracy: 0.0588 - val_loss: 4.3382
Epoch 4/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 159ms/step - accuracy: 0.0888 - loss: 4.1710 - val_accuracy: 0.0882 - val_loss: 4.1096
Epoch 5/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 146ms/step - accuracy: 0.1084 - loss: 3.8883 - val_accuracy: 0.1275 - val_loss: 3.8978
Epoch 6/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 121ms/step - accuracy: 0.1658 - loss: 3.5211 - val_accuracy: 0.1863 - val_loss: 3.7035
Epoch 7/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 6s 146ms/step - accuracy: 0.2436 - loss: 3.3270 - val_accuracy: 0.2157 - val_loss: 3.5191
Epoch 8/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 121ms/step